# Deep learning for time series

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Load data

In [ ]:
# download and unzip data
# weather timeseries dataset recorded at the weather station at the
# Max Planck Institute for Biogeochemistry in Jena, Germany.1
# In this dataset, 14 different quantities were recorded every 
# 10 minutes over several years

from zipfile import ZipFile
import os
#os.mkdir('data_folder')

uri = "https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip"
zip_path = keras.utils.get_file(origin=uri, fname= "jena_climate_2009_2016.csv.zip")
zip_file = ZipFile(zip_path)
zip_file.extractall(path='./data_folder')

csv_path = "./data_folder/jena_climate_2009_2016.csv"
raw_data = pd.read_csv(csv_path)

In [ ]:
raw_data.info()
raw_data.describe()

In [ ]:
raw_data = raw_data.drop(columns=['Date Time'])
temperature = raw_data.iloc[:,1]
plt.plot(temperature)

In [ ]:
# data recorded every 10 minutes, so 24*6=144 records per day
plt.plot(range(1440), temperature[:1440])

## Data preparation

In [ ]:
# number of samples for each split
n_train = int(0.5 * len(raw_data))
n_val = int(0.25 * len(raw_data))
n_test = len(raw_data) - n_train - n_val

print(f"train samples: {n_train}")
print(f"val samples: {n_val}")
print(f"test samples: {n_test}")


In [ ]:
# standardize data
mean = raw_data[:n_train].mean(axis=0)
raw_data -= mean
std = raw_data[:n_train].std(axis=0)
raw_data /= std

In [ ]:
# given data covering the previous five days and sampled once per hour, 
# can we predict the temperature in 24 hours?

# create a Dataset object that yields batches of data from the past five days
# along with a target temperature 24 hours in the future.

# NB
int_sequence = np.arange(10)
dummy_dataset = keras.utils.timeseries_dataset_from_array(
    data=int_sequence[:-3],
    targets=int_sequence[3:], # target is next step in the series
    sequence_length=3, # sequences 3 steps long
    batch_size=2,
)
for inputs, targets in dummy_dataset:
    for i in range(inputs.shape[0]):
        print([int(x) for x in inputs[i]], int(targets[i]))

In [ ]:
sampling_rate = 6 # sample one data point per hour
sequence_length = 120 # sequence is 5 days long -> 5*24=120 hours
delay = sampling_rate * (sequence_length + 24 - 1) # target is temperature 24 hours after the sequence
batch_size = 128

train_ds = keras.utils.timeseries_dataset_from_array(
    raw_data[:-delay],
    targets=temperature[delay:],
    sampling_rate=sampling_rate,
    sequence_length=sequence_length,
    shuffle=True,
    batch_size=batch_size,
    start_index=0,
    end_index=n_train)

val_ds = keras.utils.timeseries_dataset_from_array(
    raw_data[:-delay],
    targets=temperature[delay:],
    sampling_rate=sampling_rate,
    sequence_length=sequence_length,
    #shuffle=True,
    batch_size=batch_size,
    start_index=n_train,
    end_index=n_train + n_val)

test_ds = keras.utils.timeseries_dataset_from_array(
    raw_data[:-delay],
    targets=temperature[delay:],
    sampling_rate=sampling_rate,
    sequence_length=sequence_length,
    #shuffle=True,
    batch_size=batch_size,
    start_index=n_train + n_val)

In [ ]:
# check shapes
for samples, targets in train_ds:
    print(f"samples shape: {samples.shape}")
    print(f"targets shape: {targets.shape}")
    break

## Modeling

In [ ]:
def get_callbacks(name):
    callbacks = [
        keras.callbacks.ModelCheckpoint(filepath='./models/'+name+'.h5', save_best_only=True),
        keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=3, min_lr=1e-4)
    ]
    return callbacks

### Simple baseline

In [ ]:
# just predict temperature 24 days from now with current temperature
def evaluate_naive_method(dataset):
    total_abs_err = 0.
    samples_seen = 0
    for samples, targets in dataset:
        preds = samples[:, -1, 1] * std[1] + mean[1]
        total_abs_err += np.sum(np.abs(preds - targets))
        samples_seen += samples.shape[0]
    return total_abs_err / samples_seen

print(f"Validation MAE: {evaluate_naive_method(val_ds):.3f}")
print(f"Test MAE: {evaluate_naive_method(test_ds):.3f}")

### Simple net

In [ ]:
# build and compile
inputs = keras.Input(shape = (sequence_length, raw_data.shape[1])) # 120*14
x = layers.Flatten()(inputs)
x = layers.Dense(16, activation='relu')(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1)(x)

model = keras.Model(inputs=inputs, outputs=outputs, name='simple_dense')

model.compile(
    optimizer = keras.optimizers.RMSprop(learning_rate=0.001),
    loss = keras.losses.MeanSquaredError(),
    metrics = [keras.metrics.MeanAbsoluteError(name='mae')]
)

# training
history = model.fit(
    train_ds,
    epochs = 10,
    validation_data = val_ds,
    callbacks = get_callbacks(model.name)
)

model = keras.models.load_model('./models/'+model.name+'.h5')

# evaluation
_, val_mae  = model.evaluate(val_ds)
_, test_mae  = model.evaluate(test_ds)

print(f"Validation MAE: {val_mae:.3f}")
print(f"Test MAE: {test_mae:.3f}")

### Using 1d convolutions

In [ ]:
# IDEA
# - a temporal convnet could reuse the same representations across different days,
#   much like a spatial convnet can reuse the same representations across different 
#   locations in an image
# - 1d convnets are ideal for any sequence data that follows the translation 
#   invariance assumption (a window's behaviour is costant across the sequence)
# BUT
# - this data is only translation-invariant for a specific timescale
# - order matters, here pooling destroys it

# build and compile
inputs = keras.Input(shape = (sequence_length, raw_data.shape[1])) # 120*14
x = layers.Conv1D(8, 24, activation="relu")(inputs) # 24 hours at a time
x = layers.MaxPooling1D(2)(x)
x = layers.Conv1D(8, 12, activation="relu")(x)
x = layers.MaxPooling1D(2)(x)
x = layers.Conv1D(8, 6, activation="relu")(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1)(x)

model = keras.Model(inputs=inputs, outputs=outputs, name='simple_conv1d')

model.compile(
    optimizer = keras.optimizers.RMSprop(learning_rate=0.001),
    loss = keras.losses.MeanSquaredError(),
    metrics = [keras.metrics.MeanAbsoluteError(name='mae')]
)

# training
history = model.fit(
    train_ds,
    epochs = 10,
    validation_data = val_ds,
    callbacks = get_callbacks(model.name)
)

model = keras.models.load_model('./models/'+model.name+'.h5')

# evaluation
_, val_mae  = model.evaluate(val_ds)
_, test_mae  = model.evaluate(test_ds)

print(f"Validation MAE: {val_mae:.3f}")
print(f"Test MAE: {test_mae:.3f}")

### Recurrent nets

In [ ]:
# simple LSTM-based model

# build and compile
inputs = keras.Input(shape = (sequence_length, raw_data.shape[1])) # 120*14
x = layers.Conv1D(8, 24, activation="relu")(inputs) # 24 hours at a time
x = layers.MaxPooling1D(2)(x)
x = layers.Conv1D(8, 12, activation="relu")(x)
x = layers.MaxPooling1D(2)(x)
x = layers.Conv1D(8, 6, activation="relu")(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1)(x)

model = keras.Model(inputs=inputs, outputs=outputs, name='simple_lstm')

model.compile(
    optimizer = keras.optimizers.RMSprop(learning_rate=0.001),
    loss = keras.losses.MeanSquaredError(),
    metrics = [keras.metrics.MeanAbsoluteError(name='mae')]
)

# training
history = model.fit(
    train_ds,
    epochs = 10,
    validation_data = val_ds,
    callbacks = get_callbacks(model.name)
)

model = keras.models.load_model('./models/'+model.name+'.h5')

# evaluation
_, val_mae  = model.evaluate(val_ds)
_, test_mae  = model.evaluate(test_ds)

print(f"Validation MAE: {val_mae:.3f}")
print(f"Test MAE: {test_mae:.3f}")